
# 🧠 Advanced Regression with Neural Networks and Hyperparameter Tuning

This notebook expands on our predictive modeling of house prices using a **deeper neural network** and **custom hyperparameters** (inspired by tuning practices).

We use:
- Multiple hidden layers with customizable neuron counts
- Tunable learning rate
- GridSearchCV for hyperparameter optimization


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor


2025-07-27 17:17:39.247509: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 📊 Load and Prepare the Data

In [2]:

df = pd.read_csv("house_prediction.csv", delim_whitespace=True, header=None)
df.columns = [
    'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM',
    'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE'
]

X = df.drop("PRICE", axis=1)
y = df["PRICE"]

scaler_x = StandardScaler()
X_scaled = scaler_x.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42
)


/var/folders/jt/sgp4392j1ysgvdd5dk6c99k00000gp/T/ipykernel_69947/1051507783.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("house_prediction.csv", delim_whitespace=True, header=None)


## 🏗️ Define Model with Custom Hyperparameters

In [3]:

def create_model(activation='relu', lr=0.001, neurons1=128, neurons2=64, neurons3=128):
    model = Sequential()
    model.add(Dense(neurons1, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(neurons2, activation=activation))
    model.add(Dense(neurons3, activation=activation))
    model.add(Dense(1, activation='linear'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model


## 🔍 Tune with GridSearchCV

In [5]:
def create_model(activation='relu', neurons1=64, neurons2=32, neurons3=64, lr=0.001):
    model = Sequential()
    model.add(Dense(neurons1, activation=activation, input_shape=(X_train.shape[1],)))
    model.add(Dense(neurons2, activation=activation))
    model.add(Dense(neurons3, activation=activation))
    model.add(Dense(1))  # saída para regressão

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mse')
    return model

model = KerasRegressor(model=create_model, verbose=0)

param_grid = {
    'model__activation': ['relu', 'tanh'],
    'model__neurons1': [64, 128],
    'model__neurons2': [32, 64],
    'model__neurons3': [64, 128],
    'model__lr': [0.001, 0.005],
    'batch_size': [16, 32],
    'epochs': [50]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    scoring='neg_mean_squared_error', cv=3)

grid_result = grid.fit(X_train, y_train)


/Users/lpn/Documents/GitHub/CODEVA_project/ml_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/lpn/Documents/GitHub/CODEVA_project/ml_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/lpn/Documents/GitHub/CODEVA_project/ml_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as 

/Users/lpn/Documents/GitHub/CODEVA_project/ml_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/lpn/Documents/GitHub/CODEVA_project/ml_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/lpn/Documents/GitHub/CODEVA_project/ml_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as 

## ✅ Evaluate Best Model

In [6]:

best_model = grid_result.best_estimator_
y_pred_scaled = best_model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_test_original = scaler_y.inverse_transform(y_test)

mse = mean_squared_error(y_test_original, y_pred)
r2 = r2_score(y_test_original, y_pred)

print("Best Params:", grid_result.best_params_)
print(f"Test MSE: {mse:.2f}")
print(f"Test R²: {r2:.2f}")


Best Params: {'batch_size': 32, 'epochs': 50, 'model__activation': 'relu', 'model__lr': 0.005, 'model__neurons1': 128, 'model__neurons2': 64, 'model__neurons3': 128}
Test MSE: 12.33
Test R²: 0.83



### 📌 Summary

- Deep neural network with 3 hidden layers and tunable parameters.
- GridSearchCV optimized neuron counts, activation, learning rate, and batch size.
- Evaluated using **MSE** and **R²**.

> This notebook showcases a more flexible and powerful modeling pipeline using deep learning.
